# Fresh and rotten fruits recognition

We will train a new model that is able to recognize fresh and rotten fruit. We would like to train our model to be at least 92% accurate on the validation dataset.

The dataset comes from Kaggle. You can download it [here](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification). This dataset is not in Date foler because it is too large. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges and rotten bananas. This means that our model will require an output layer of 6 neurons to do the categorization successfully. We will also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

## Load the ImageNet Base Model

We will start with model pretrained on ImageNet. We will load the model with the correct weights, set an input shape and choose to remove the last layer of the model.

Notice: The images have three dimensions: a height and with and a number of channels. Because these pictures ale in color, there will be three channels for red, green and blue.

In [1]:
from tensorflow import keras

In [2]:
base_model = keras.applications.VGG16(weights='imagenet',
                                           input_shape=(224, 224, 3),
                                           include_top=False)

## Freze base model

Next we will freeze the base model so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [3]:
base_model.trainable = False

## Add layers to model

Now we will add layers to the pretrained model. We should pay attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [4]:
# Create inpts
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation='softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile the model

Now we will compile the model with loss and metric options. Remember that we are training on a number of different categories, rather than a binary classification problem.

In [6]:
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

## Augument the Data

We can try to augment the data to improve the dataset. It may be helpful to get 92% accuracy.

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
datagen = ImageDataGenerator(rotation_range=10,
                            zoom_range=0.1,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=False)

## Load dataset

Next, we will load the train and validation datasets. We have to pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model we have created).

In [9]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('Data/fruits/train/',
                                   target_size=(224, 224),
                                   color_mode='rgb',
                                   class_mode='categorical')

# load and iterate validation dataset
test_it = datagen.flow_from_directory('Data/fruits/test/',
                                   target_size=(224, 224),
                                   color_mode='rgb',
                                   class_mode='categorical')

Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


## Train the model

Now it is time to train the model. We have to pass the `train` and `valid` iterators into the `fit` function, as well as setting our desired number of epochs.

In [11]:
model.fit(train_it,
         validation_data=test_it,
         steps_per_epoch=len(train_it)/train_it.batch_size,
         validation_steps=len(test_it)/test_it.batch_size,
         epochs=15)

Epoch 1/15
10/10 [==============================] - 55s 5s/step - loss: 3.0986 - categorical_accuracy: 0.3551 - val_loss: 2.5571 - val_categorical_accuracy: 0.4583
Epoch 2/15
10/10 [==============================] - 56s 5s/step - loss: 1.8669 - categorical_accuracy: 0.5284 - val_loss: 1.9860 - val_categorical_accuracy: 0.5521
Epoch 3/15
10/10 [==============================] - 58s 6s/step - loss: 1.4790 - categorical_accuracy: 0.6108 - val_loss: 1.3753 - val_categorical_accuracy: 0.6667
Epoch 4/15
10/10 [==============================] - 59s 6s/step - loss: 1.3377 - categorical_accuracy: 0.6591 - val_loss: 1.3446 - val_categorical_accuracy: 0.6667
Epoch 5/15
10/10 [==============================] - 58s 5s/step - loss: 1.0725 - categorical_accuracy: 0.7017 - val_loss: 0.8836 - val_categorical_accuracy: 0.7812
Epoch 6/15
10/10 [==============================] - 57s 5s/step - loss: 1.0022 - categorical_accuracy: 0.7244 - val_loss: 0.5910 - val_categorical_accuracy: 0.8125
Epoch 7/15
10/10

## Unfreeze model for fine tuning

If our model have not reached 92% validation accuracy we can makie fine tunning: unfreeze the base model and compile the model with a low learning rate.

In [12]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.00001),
             loss = keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=[keras.metrics.BinaryAccuracy()])

In [14]:
model.fit(train_it,
         validation_data=test_it,
         steps_per_epoch=len(train_it)/train_it.batch_size,
         validation_steps=len(test_it)/test_it.batch_size,
         epochs=5)

Epoch 1/5
10/10 [==============================] - 190s 18s/step - loss: 0.1493 - binary_accuracy: 0.9640 - val_loss: 0.1383 - val_binary_accuracy: 0.9722
Epoch 2/5
10/10 [==============================] - 190s 18s/step - loss: 0.1122 - binary_accuracy: 0.9749 - val_loss: 0.0580 - val_binary_accuracy: 0.9913
Epoch 3/5
10/10 [==============================] - 186s 18s/step - loss: 0.0826 - binary_accuracy: 0.9790 - val_loss: 0.0797 - val_binary_accuracy: 0.9688
Epoch 4/5
10/10 [==============================] - 193s 18s/step - loss: 0.0933 - binary_accuracy: 0.9777 - val_loss: 0.0581 - val_binary_accuracy: 0.9896
Epoch 5/5
10/10 [==============================] - 190s 18s/step - loss: 0.0606 - binary_accuracy: 0.9896 - val_loss: 0.1200 - val_binary_accuracy: 0.9688


## Evaluate the model

Now we have a model that has a validation accuracy higher than 92% and we can evaluate it. The evaluate function will return tuple, where the first value is our loss and the second values is our accuracy.

In [15]:
model.evaluate(test_it, steps=len(test_it)/test_it.batch_size)

2/2 [==============================] - 11s 4s/step - loss: 0.1242 - binary_accuracy: 0.9705


[0.12415876239538193, 0.9704861640930176]

In [16]:
# Clear the GPU memory
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}